In [1]:
# Get all the players from Sleeper and store the pertinent information in a SQLite database
import requests
import sqlite3
import json
from sleeper import get_all_players, get_users_in_league, get_all_rosters, get_matchups
from manage import clean_player_data
from config import LEAGUE_LIST
import pandas as pd
SLEEPER_API = "https://api.sleeper.app/v1/players/nfl"

%load_ext autoreload
%autoreload 2

# Gather all the players from Sleeper
This should only need to be done once, but it's a good idea to check for new players every now and then. This notebook will gather all the players from Sleeper and put them in the database.

In [2]:
# Get all the players from Sleeper
players_json = get_all_players()
players_df = clean_player_data(players_json)

# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
players_df.to_sql('players', conn, if_exists='replace', index=True)
conn.commit()
conn.close()

# Insert the league id with the league name into the db
This is a manual process of keeping this up to date

In [3]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS league")

# Convert to a DataFrame
df = pd.DataFrame(list(LEAGUE_LIST.items()), columns=['name', 'league_id'])
df.to_sql('league', conn, if_exists='append', index=False)


8

# Insert the coaches into the database

In [4]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS coaches")
# Get all the coaches in each of the leagues
for league in LEAGUE_LIST:
    coaches_df = get_users_in_league(LEAGUE_LIST[league])
    coaches_df.to_sql('coaches', conn, if_exists='append', index=False)

# Update the team names if they aren't already set by combining the display name and Team like "Team" + Display Name from the row
conn.execute("UPDATE coaches SET team_name = 'Team ' + display_name WHERE team_name IS NULL")

# Close the connection
conn.close()

# Now get the drafts from every league and put these in the database
We need to have the rosters so that we can deal with matchups each week.  Each match up has a roster_id associated with it, so we need to map owners to rosters to determine the win-loss of each owner.

In [80]:
# Get the drafts for each league
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
conn.execute("DROP TABLE IF EXISTS rosters")

for league in LEAGUE_LIST:
    rosters = get_all_rosters(LEAGUE_LIST[league])
        
    # Expand the players columns into individual rows keeping just the player_id, the owner_id, and league_id
    rosters_df = rosters.explode('players')[['roster_id','players','owner_id','league_id']].rename(columns={'players':'player_id'})
    
    # Move the roster_id to a new column
    # rosters_df['roster_id'] = rosters_df.index
    
    # Create a new roster index made up of the league_id and the roster_id
    rosters_df.index = rosters_df['league_id'].astype(str) + "_" + rosters_df['roster_id'].astype(str)    
        
    rosters_df.to_sql('rosters', conn, if_exists='append', index=False)
# Close the connection
conn.close()


# Load Bye Weeks into the Database

In [6]:
# Load bye weeks into the SQLite database
conn = sqlite3.connect('../data/league_db.db')
conn.execute("DROP TABLE IF EXISTS team")
# Open the file with the bye weeks
with open('../data/nfl_bye_weeks_2024.csv', 'r') as file:
    # Read the data into a dataframe
    bye_weeks = pd.read_csv(file)
    # Insert the data into the database
    bye_weeks.to_sql('team', conn, if_exists='replace', index=False)
conn.close()

erDiagram
    PLAYERS {
        int player_id PK
        int rotoworld_id
        float weight
        string practice_description
        date birth_date
        string team_abbr
        int fantasy_data_id
        bool active
        int opta_id
        int age
        int espn_id
        string first_name
        string hashtag
        int depth_chart_order
        string status
        int pandascore_id
        int stats_id
        int swish_id
        string search_first_name
        string search_last_name
        int oddsjam_id
        string college
        string last_name
        string team
        string full_name
        string search_full_name
        float height
        string fantasy_positions
        string practice_participation
        string position
        int years_exp
        int number
        string gsis_id
        string depth_chart_position
        int search_rank
        string sport
    }

    LEAGUE {
        int league_id PK
        string name
    }

    COACHES {
        int user_id PK
        string display_name
        bool is_owner
        int league_id FK
        string team_name
        date team_name_update
        bool allow_sms
    }

    ROSTERS {
        int roster_id PK
        int player_id FK
        int owner_id FK
        int league_id FK
    }

    TEAM {
        string Team PK
        int Bye_Week
    }

    OWNER
    
    PLAYERS ||--o{ ROSTERS : "is part of"
    COACHES ||--o{ ROSTERS : "owns"
    LEAGUE ||--o{ ROSTERS : "includes"
    LEAGUE ||--o{ COACHES : "has"
    TEAM ||--o{ PLAYERS : "belongs to"

# Load weekly matchups into the database
Here we are going to load the weekly matchups into the database.  This will allow us to determine the win-loss record of each owner.

**NOTE:** Change the week before entering the data into the database

In [2]:
def update_weekly_matchups(week = 1, drop_table = False):
    conn = sqlite3.connect('../data/league_db.db')
    if drop_table:
        conn.execute("DROP TABLE IF EXISTS matchups")
    
    def update_matchups(league_id, week):
        df_matchup = get_matchups(league_id, week)
        df_matchup[['league_id','week']] = league_id, week
        df_matchup = df_matchup[['league_id','week','points','roster_id','matchup_id']]
        #print(df_matchup)
        df_matchup.to_sql('matchups', conn, if_exists='append', index=False)
        
    # For each league
    for league in LEAGUE_LIST:
        league_id = LEAGUE_LIST[league]
        if drop_table:
            for w in range(1, week + 1):
                update_matchups(league_id, w)
        else:
            update_matchups(league_id, week)
        
    conn.close()       

# Utility Functions
So here we will define some utility functions that we will use throughout the notebook.  These are useful once all the tables are loaded up.

In [3]:
# Run a sqlite query
def run_query(q, cols=None):
    conn = sqlite3.connect('../data/league_db.db')
    rows = conn.execute(q).fetchall()
    if cols is None:
        cols = [desc[0] for desc in conn.execute(q).description]
    df = pd.DataFrame(rows, columns=cols)
    return df

def run_query_scalar(q):
    conn = sqlite3.connect('../data/league_db.db')
    rows = conn.execute(q).fetchone()
    return rows[0]

def get_column_names(table_name):
    # Connect to the SQLite3 database
    conn = sqlite3.connect('../data/league_db.db')
    cursor = conn.cursor()
    
    # Execute PRAGMA table_info to get the column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    
    # Fetch all results
    columns_info = cursor.fetchall()
    
    # Extract column names
    column_names = [info[1] for info in columns_info]
    
    # Close the connection
    conn.close()
    
    return column_names

def table_names():
    # Get the list of tables in teh sqlite database
    conn = sqlite3.connect('../data/league_db.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return tables


for table in table_names():
    print(f'{table[0]}: {get_column_names(table[0])}')
    
#run_query('Select p.full_name, r.league_id, p.player_id '
                    #'FROM players p join rosters r on r.player_id = p.player_id WHERE full_name like "%Davante Adams%"')


players: ['player_id', 'stats_id', 'oddsjam_id', 'search_rank', 'team_abbr', 'search_full_name', 'gsis_id', 'first_name', 'age', 'search_first_name', 'team', 'active', 'last_name', 'rotoworld_id', 'search_last_name', 'espn_id', 'weight', 'college', 'opta_id', 'full_name', 'status', 'swish_id', 'yahoo_id', 'fantasy_positions', 'rotowire_id', 'number', 'fantasy_data_id', 'hashtag', 'years_exp', 'pandascore_id', 'height', 'practice_description', 'birth_date', 'sportradar_id', 'position', 'practice_participation', 'sport', 'depth_chart_position', 'depth_chart_order']
league: ['name', 'league_id']
coaches: ['display_name', 'is_owner', 'league_id', 'user_id', 'team_name', 'team_name_update', 'allow_sms']
rosters: ['roster_id', 'player_id', 'owner_id', 'league_id']
matchups: ['league_id', 'week', 'points', 'roster_id', 'matchup_id']


# Create spreadsheet of rostered players

In [8]:
conn = sqlite3.connect('../data/league_db.db')
rows = conn.execute(f'Select p.full_name, p.espn_id,p.rotowire_id, p.sportradar_id,p.yahoo_id, p.college, p.years_exp, p.team, p.position, c.display_name, c.league_id, t."Bye Week"'
                    'FROM players p '
                    'JOIN rosters r on p.player_id = r.player_id '
                    'JOIN coaches c on r.owner_id = c.user_id and r.league_id = c.league_id '
                    'JOIN team t on p.team=t.team').fetchall()
# turn the rows into a dataframe
df = pd.DataFrame(rows, columns=['full_name','espn_id','rotowire_id', 'sportradar_id','yahoo_id','college', 'years_experience','team', 'position', 'coach','league_id','bye'])
conn.close()

# Create a reverse dictionary to map IDs to names
reverse_LEAGUE_LIST = {str(v): k for k, v in LEAGUE_LIST.items()}
# Ensure the league_id column is of type string
df['league_id'] = df['league_id'].astype(str)
df['espn_id']=df['espn_id'].fillna(-1).astype(int)
df['rotowire_id']=df['rotowire_id'].fillna(-1).astype(int)
df['yahoo_id']=df['yahoo_id'].fillna(-1).astype(int)

# Now replace the league name with the league id
df['league_name'] = df['league_id'].map(reverse_LEAGUE_LIST)
df.drop(columns=['league_id'], inplace=True)

# Output the data to a csv file
df.to_csv('../data/players_rostered.csv', index=False)
df

,full_name,espn_id,rotowire_id,sportradar_id,yahoo_id,college,years_experience,team,position,coach,bye,league_name
0,Jonathan Mingo,-1,16937,6cf0c82d-87e4-4cfc-8475-77667ed92796,-1,Ole Miss,1.0,CAR,WR,ChiJV,11,MainLeague
1,Rashee Rice,-1,16963,cbd73c09-da07-48b0-a0d6-f248dfa631b6,-1,SMU,1.0,KC,WR,ChiJV,6,MainLeague
2,Roschon Johnson,-1,16772,e3c414c0-be8c-4807-af9c-f91d89c83e60,-1,Texas,1.0,CHI,RB,ChiJV,7,MainLeague
3,Brenden Rice,-1,17756,5c33558d-02fb-4c99-88d3-37083472c19f,-1,USC,0.0,LAC,WR,ChiJV,5,MainLeague
4,Luke McCaffrey,-1,17820,5cfd82f8-9e73-48d7-914e-9051ff52d67d,-1,Rice,0.0,WAS,WR,ChiJV,14,MainLeague
...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Kenneth Walker,-1,15909,22ee9bac-a64c-4d44-94fc-51d775465b3b,-1,Michigan State,2.0,SEA,RB,WestCoastNonsense,10,Quickdraft 2
2022,Brock Purdy,-1,15834,d5aef708-ad61-4ab8-a637-62ff96e92040,-1,Iowa State,2.0,SF,QB,WestCoastNonsense,9,Quickdraft 2
2023,Anthony Richardson,-1,16653,5fc5b4a6-a583-4345-a810-4f982204ea5e,-1,Florida,1.0,IND,QB,WestCoastNonsense,14,Quickdraft 2
2024,C.J. Stroud,-1,16886,7e91086f-9ca4-4bd9-a099-b88b2f37e6c6,-1,Ohio State,1.0,HOU,QB,WestCoastNonsense,14,Quickdraft 2


# Create the weekly matchups csv
Once we have matchups updated in the database, then we can put together the weekly matchups csv.  This will be used to determine the win-loss record of each owner.  We also have to deal with the median in each league.  I'm not sure if we'll do this with sql or python yet.

In [4]:
latest_week = run_query_scalar("select max(week) from matchups")+1
print(f'Updating matchups for week {latest_week}')
update_weekly_matchups(latest_week, drop_table=False)

Updating matchups for week 5


Once we have the matchups in the database, then we need to update the csv file.  This will be done in the next cell.

In [5]:

df_outcomes = run_query('''WITH roster_coaches AS (
    SELECT 
        r.roster_id, 
        r.league_id,
        l.name as league_name,
        c.display_name as coach_name,
        c.user_id
    FROM 
        rosters r JOIN coaches c ON r.owner_id = c.user_id and r.league_id = c.league_id
        JOIN league l on r.league_id = l.league_id
    GROUP BY 
        r.roster_id, r.league_id, l.name, c.display_name, c.user_id -- Aggregate to one row per roster
)
SELECT r1.league_name, m1.week, r1.coach_name as 'Winner', r2.coach_name as 'Loser', m1.points, m2.points
    FROM matchups m1 
    JOIN matchups m2 on m1.matchup_id = m2.matchup_id and m1.week = m2.week and m1.league_id = m2.league_id and m1.roster_id != m2.roster_id
    JOIN roster_coaches r1 on m1.roster_id = r1.roster_id and m1.league_id = r1.league_id
    JOIN roster_coaches r2 on m2.roster_id = r2.roster_id and m2.league_id = r2.league_id
    WHERE m1.points > m2.points''', 
    cols=['League','Week','Winner','Loser','Winner Pts','Loser Pts'])

print(df_outcomes)


         League  Week     Winner              Loser  Winner Pts  Loser Pts
0    MainLeague     3      ChiJV          ConorDeLa      182.92     101.92
1    MainLeague     5      ChiJV         Mschmoopie      183.00     131.08
2    MainLeague     1      ChiJV            tford51      148.72     115.78
3    MainLeague     4      ChiJV         ProfessorD      158.10     130.18
4    MainLeague     2      ChiJV           plfinken      166.48     126.28
..          ...   ...        ...                ...         ...        ...
215         BB3     1  JeffMyers              aledo      162.16     153.68
216         BB3     3  JeffMyers          gpeisert1      167.36     133.26
217         BB4     1   msbosway             jnappy      168.74     155.62
218         BB4     3   msbosway  WestCoastNonsense      150.26     120.42
219         BB5     1    Fordo45              aledo      148.70     117.74

[220 rows x 6 columns]


Now we run the query to get the actual matchups into a csv file so that it can be processed by the jscript.

In [6]:
# Calculate the median for each league
df_median_points = run_query('''select league_id, week, points from matchups''').groupby(['league_id','week']).median().reset_index()

df_team_points = run_query('''WITH roster_coaches AS (
    SELECT 
        r.roster_id, 
        r.league_id,
        l.name as league_name,
        c.display_name as coach_name,
        c.user_id
    FROM 
        rosters r JOIN coaches c ON r.owner_id = c.user_id and r.league_id = c.league_id
        JOIN league l on r.league_id = l.league_id
    GROUP BY 
        r.roster_id, r.league_id, l.name, c.display_name, c.user_id -- Aggregate to one row per roster
)
SELECT r.league_id, r.league_name, m.week, m.roster_id, r.coach_name, m.points 
FROM matchups m 
    JOIN roster_coaches r on m.roster_id = r.roster_id and m.league_id = r.league_id''')

df_team_points['league_id'] = df_team_points['league_id'].astype(int)

df_combined = pd.merge(df_team_points, df_median_points, on=['league_id','week'], suffixes=('_team','_median'),how='inner')
df_combined['Winner']=df_combined.apply(lambda row: row['coach_name'] if row['points_team']> row['points_median'] else 'MEDIAN', axis=1)
df_combined['Loser']=df_combined.apply(lambda row: row['coach_name'] if row['points_team']<= row['points_median'] else 'MEDIAN', axis=1)
df_output= df_combined[['league_name','week','Winner','Loser']].rename(columns={'league_name':'League','week':'Week'})
pd.concat([df_outcomes,df_output], axis=0).to_csv('../data/league_outcomes.csv', index=False)
